In [ ]:
import time
import httpx
import lxml.html
import json
import csv
import os
import pandas as pd
from zillow_utils import parse_script_content, save_listings_to_csv

headers = {
    "method": "GET",
    "scheme": "https",
    "authority": "www.zillow.com",
    "path": "/apartments/elk-grove-village-il/willow-crossing-apartments/5Xt94L/",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-US,en;q=0.9",
    "Connection": "keep-alive",
    "Cookie": "sessionid=abc123",
    "Host": "www.zillow.com",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "none",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.6 Safari/605.1.15"
}



def fetch_page(url: str) -> str:
    """
    Fetches the content of a webpage given a URL using httpx.
    """

    with httpx.Client(headers=headers, follow_redirects=True) as client:
        response = client.get(url)
        response.raise_for_status()  # Raise an exception for HTTP errors
        return response.text


def extract_listings(json_data: dict):
    """
    Extracts the list of property listings from the parsed JSON data.
    """
    try:
        partial_extract = json_data['props']['pageProps']['componentProps']['initialReduxState']['gdp']['building']
        listings = partial_extract.get('floorPlans') 
        if not listings:
            listings= partial_extract.get('ungroupedUnits')
        return listings
    except KeyError:
        return None


def get_listing_info(listings:list):
    """
    Extract data from a specific listing and returns a clean dictionary
    """

    results = []
    for listing in listings:
        listingType = listing.get('listingType')
        beds = listing.get("beds")
        baths = listing.get("baths")
        sqft = listing.get("sqft")
        min_price = listing.get("minPrice")
        max_price = listing.get("maxPrice")
        
        if listing.get("price"):
            price = listing.get('price')

            if min_price is None:
                min_price = price
            
            if max_price is None:
                max_price = price

        if listingType == 'FOR_RENT' or listingType is None:
            if min_price != max_price:
                results.append({
                    "price": max_price,
                    "beds": beds,
                    "baths": baths,
                    "sqft": sqft,
                    "type": listingType,
                })

            results.append({
                "price": min_price,
                "beds": beds,
                "baths": baths,
                "sqft": sqft,
                "type": listingType,
            })
    return results



#url="https://www.zillow.com/b/726-w-19th-pl-chicago-il-9Rz296/"
url="https://www.zillow.com/homedetails/2000-S-Desplaines-St-Chicago-IL-60616/158858755_zpid/"
detail_link = str(url)
html = fetch_page(detail_link)
json_data = parse_script_content(html)
if json_data:
    listings = extract_listings(json_data)
    print("Fetching URl:", url)

json_data




{'props': {'encodedZuid': '',
  'guid': '8bdf0b81-3c40-43a5-8d65-268c03a3443d',
  'isBot': False,
  'deviceType': 'desktop',
  'zuid': '',
  'pageProps': {'moduleVersion': 'not-for-sale-page-sub-app.main.8415713.5fec9557',
   'componentProps': {'zpid': 158858755,
    'altId': None,
    'gdpClientCache': '{"NotForSaleShopperPlatformFullRenderQuery{\\"zpid\\":158858755,\\"altId\\":null,\\"deviceType\\":\\"desktop\\"}":{"property":{"zpid":158858755,"listingSource":null,"listingAccount":null,"ownerAccount":null,"lfaViewPropertyPageUrl":null,"listingOwnerConfigIDs":null,"postingPresentationTypes":null,"maloneId":"2104019263","city":"Chicago","state":"IL","homeStatus":"OTHER","address":{"streetAddress":"2000 S Desplaines St","city":"Chicago","state":"IL","zipcode":"60616","neighborhood":null,"community":null,"subdivision":null},"isListingClaimedByCurrentSignedInUser":false,"isCurrentSignedInAgentResponsible":false,"bedrooms":null,"bathrooms":0,"price":88200,"yearBuilt":null,"streetAddress":"